# PyQGIS Notebook Examples

This notebook demonstrates how to use PyQGIS within the QGIS Notebook plugin.

**Pre-imported modules:** `iface`, `QgsProject`, `QgsVectorLayer`, `QgsRasterLayer`, `QgsGeometry`, `QgsFeature`, `QgsPointXY`, `processing`, `os`, `sys`

## 1. Adding Basemaps

Add OpenStreetMap as a basemap layer using XYZ tiles.

In [ ]:
# Add OpenStreetMap basemap
osm_url = "type=xyz&url=https://tile.openstreetmap.org/{z}/{x}/{y}.png"
iface.addRasterLayer(osm_url, "OpenStreetMap", "wms")

## 2. Loading Vector Data

Load a GeoJSON file from a remote URL.

In [ ]:
# Load world cities from GeoJSON
cities_url = (
    "https://github.com/opengeos/datasets/releases/download/world/world_cities.geojson"
)
cities_layer = iface.addVectorLayer(cities_url, "World Cities", "ogr")

# Check if layer loaded successfully
if cities_layer and cities_layer.isValid():
    print(f"Loaded {cities_layer.featureCount()} cities")
else:
    print("Failed to load layer")

## 3. Loading Raster Data

Load a Landsat satellite image from a remote URL.

In [ ]:
# Load Landsat raster
landsat_url = (
    "https://github.com/opengeos/datasets/releases/download/raster/landsat.tif"
)
landsat_layer = iface.addRasterLayer(landsat_url, "Landsat", "gdal")

if landsat_layer and landsat_layer.isValid():
    print(f"Raster size: {landsat_layer.width()} x {landsat_layer.height()}")
    print(f"Band count: {landsat_layer.bandCount()}")

## 4. Navigating the Map

Zoom to layer extent and refresh the canvas.

In [ ]:
# Zoom to raster extent
if landsat_layer and landsat_layer.isValid():
    iface.mapCanvas().setExtent(landsat_layer.extent())
    iface.mapCanvas().refresh()
    print("Zoomed to Landsat extent")

## 5. Customizing Raster Visualization

Apply a custom band combination with contrast stretching.

In [ ]:
from qgis.core import QgsMultiBandColorRenderer, QgsContrastEnhancement

# Get the Landsat layer
layers = QgsProject.instance().mapLayersByName("Landsat")
if layers:
    layer = layers[0]

    # Create RGB renderer (NIR-Red-Green false color)
    renderer = QgsMultiBandColorRenderer(
        layer.dataProvider(), redBand=4, greenBand=3, blueBand=2  # NIR  # Red  # Green
    )

    # Apply contrast stretch
    def create_stretch(band, vmin, vmax):
        ce = QgsContrastEnhancement(layer.dataProvider().dataType(band))
        ce.setMinimumValue(vmin)
        ce.setMaximumValue(vmax)
        ce.setContrastEnhancementAlgorithm(
            QgsContrastEnhancement.StretchToMinimumMaximum
        )
        return ce

    renderer.setRedContrastEnhancement(create_stretch(4, 0, 100))
    renderer.setGreenContrastEnhancement(create_stretch(3, 0, 100))
    renderer.setBlueContrastEnhancement(create_stretch(2, 0, 100))

    layer.setRenderer(renderer)
    layer.triggerRepaint()
    print("Applied false color composite")

## 6. Working with Features

Query and iterate through vector features.

In [ ]:
# Get the cities layer
layers = QgsProject.instance().mapLayersByName("World Cities")
if layers:
    layer = layers[0]

    # Print field names
    fields = [f.name() for f in layer.fields()]
    print(f"Fields: {fields}")

    # Get first 5 features
    print("\nFirst 5 cities:")
    for i, feature in enumerate(layer.getFeatures()):
        if i >= 5:
            break
        name = (
            feature.attribute("name") if "name" in fields else f"Feature {feature.id()}"
        )
        print(f"  - {name}")

## 7. Creating a Memory Layer

Create a new vector layer in memory and add features.

In [ ]:
from qgis.core import QgsField
from qgis.PyQt.QtCore import QVariant

# Create a point layer in memory
mem_layer = QgsVectorLayer("Point?crs=EPSG:4326", "My Points", "memory")
provider = mem_layer.dataProvider()

# Add fields
provider.addAttributes(
    [QgsField("name", QVariant.String), QgsField("value", QVariant.Double)]
)
mem_layer.updateFields()

# Add some features
features = []
locations = [
    ("New York", -74.006, 40.7128, 8.3),
    ("London", -0.1276, 51.5074, 8.9),
    ("Tokyo", 139.6917, 35.6895, 13.9),
    ("Sydney", 151.2093, -33.8688, 5.3),
]

for name, lon, lat, value in locations:
    feat = QgsFeature()
    feat.setGeometry(QgsGeometry.fromPointXY(QgsPointXY(lon, lat)))
    feat.setAttributes([name, value])
    features.append(feat)

provider.addFeatures(features)
QgsProject.instance().addMapLayer(mem_layer)
print(f"Created layer with {mem_layer.featureCount()} features")

## 8. Using Processing Algorithms

Run QGIS processing algorithms.

In [ ]:
# List available processing providers
for provider in QgsApplication.processingRegistry().providers():
    print(f"{provider.name()}: {len(provider.algorithms())} algorithms")

In [ ]:
# Example: Buffer the points layer
layers = QgsProject.instance().mapLayersByName("My Points")
if layers:
    result = processing.run(
        "native:buffer",
        {
            "INPUT": layers[0],
            "DISTANCE": 1,  # 1 degree buffer
            "SEGMENTS": 16,
            "OUTPUT": "memory:",
        },
    )

    buffer_layer = result["OUTPUT"]
    buffer_layer.setName("Point Buffers")
    QgsProject.instance().addMapLayer(buffer_layer)
    print("Created buffer layer")

## 9. Cleaning Up

Remove all layers from the project.

In [ ]:
QgsProject.instance().removeAllMapLayers()
iface.mapCanvas().refresh()
print("Removed all layers")